In [159]:
# Import modules
import pandas as pd
from ast import literal_eval
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import requests
import json
from pandas import json_normalize

In [44]:
# Load dataset
links_df = pd.read_csv('64k_links.csv')
movies_df = pd.read_csv('64k_movies.csv')

In [45]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [51]:
# need to extract the years so we can split our data based on a specific time period
year = movies_df['title'].str.rsplit('(', n=1, expand=True)

In [56]:
year = year[1].str.rsplit(')', n=1, expand=True)

In [62]:
year.head()

,0,1
0,1995,
1,1995,
2,1995,
3,1995,
4,1995,


In [63]:
year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       62016 non-null  object
 1   1       62016 non-null  object
dtypes: object(2)
memory usage: 975.5+ KB


In [64]:
movies_df['year'] = year[0]

In [65]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [66]:
# will need to delete some of these years later
movies_df.year.unique()

array(['1995', '1994', '1996', '1976', '1992', '1988', '1967', '1993',
       '1964', '1977', '1965', '1982', '1990', '1991', '1989', '1937',
       '1940', '1969', '1981', '1973', '1970', '1960', '1955', '1959',
       '1968', '1980', '1975', '1986', '1948', '1943', '1950', '1946',
       '1987', '1997', '1974', '1956', '1958', '1949', '1972', '1998',
       '1933', '1952', '1951', '1957', '1961', '1954', '1934', '1944',
       '1963', '1942', '1941', '1953', '1939', '1947', '1945', '1938',
       '1935', '1936', '1926', '1932', '1985', '1979', '1971', '1978',
       '1966', '1962', '1983', '1984', '1931', '1922', '1999', '1927',
       '1929', '1930', '1928', '1925', '1914', '2000', '1919', '1923',
       '1920', '1918', '1921', '2001', '1924', '2002', '2003', '1915',
       '2004', '1916', '1917', '2005', '2006', '1902', '1903', '2007',
       '2008', '2009', '1912', '2010', 'Das Millionenspiel', '1913',
       '2011', '1898', '1899', '1894', '2012', '1910', '2013', '1983)',
       

In [67]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [68]:
movies_df

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
62418,209157,We (2018),Drama,2018
62419,209159,Window of the Soul (2001),Documentary,2001
62420,209163,Bad Poems (2018),Comedy|Drama,2018
62421,209169,A Girl Thing (2001),(no genres listed),2001


In [70]:
movies = pd.merge(links_df, movies_df,
        left_on='movieId',
        right_on='movieId')

In [73]:
movies = movies.drop(columns=['title','genres'])

In [76]:
movies.year.unique()

array(['1995', '1994', '1996', '1976', '1992', '1988', '1967', '1993',
       '1964', '1977', '1965', '1982', '1990', '1991', '1989', '1937',
       '1940', '1969', '1981', '1973', '1970', '1960', '1955', '1959',
       '1968', '1980', '1975', '1986', '1948', '1943', '1950', '1946',
       '1987', '1997', '1974', '1956', '1958', '1949', '1972', '1998',
       '1933', '1952', '1951', '1957', '1961', '1954', '1934', '1944',
       '1963', '1942', '1941', '1953', '1939', '1947', '1945', '1938',
       '1935', '1936', '1926', '1932', '1985', '1979', '1971', '1978',
       '1966', '1962', '1983', '1984', '1931', '1922', '1999', '1927',
       '1929', '1930', '1928', '1925', '1914', '2000', '1919', '1923',
       '1920', '1918', '1921', '2001', '1924', '2002', '2003', '1915',
       '2004', '1916', '1917', '2005', '2006', '1902', '1903', '2007',
       '2008', '2009', '1912', '2010', 'Das Millionenspiel', '1913',
       '2011', '1898', '1899', '1894', '2012', '1910', '2013', '1983)',
       

In [97]:
movies['year'].str.isnumeric()

0        True
1        True
2        True
3        True
4        True
         ... 
62418    True
62419    True
62420    True
62421    True
62422    True
Name: year, Length: 62423, dtype: object

In [101]:
# dropping any non-numeric rows from the dataframe
movies = movies[pd.to_numeric(movies['year'], errors='coerce').notnull()]

In [106]:
movies['year'] = movies['year'].astype(int)

C:\Users\annie\AppData\Local\Temp\ipykernel_17848\3517707740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['year'] = movies['year'].astype(int)


In [108]:
movies.head()

,movieId,imdbId,tmdbId,year
0,1,114709,862.0,1995
1,2,113497,8844.0,1995
2,3,113228,15602.0,1995
3,4,114885,31357.0,1995
4,5,113041,11862.0,1995


In [112]:
# only taking movies from after 2007 because that's when netflix streaming started
movies = movies.loc[movies['year'] >= 2007]

In [148]:
tmdb_id_df = movies['tmdbId'].str.split('.', expand=True)

In [151]:
movies['tmdbId'] = tmdb_id_df[0]

In [153]:
movies.head()

,movieId,imdbId,tmdbId,year
11272,49910,463998,1646,2007
11294,50149,308353,5393,2007
11295,50151,763840,14171,2007
11296,50153,462229,14396,2007
11297,50158,775539,1931,2007


In [154]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25343 entries, 11272 to 62420
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  25343 non-null  int64 
 1   imdbId   25343 non-null  int64 
 2   tmdbId   25343 non-null  object
 3   year     25343 non-null  int32 
dtypes: int32(1), int64(2), object(1)
memory usage: 891.0+ KB


In [155]:
# exporting cleaned dataframe
movies.to_csv('movies_full.csv', index=False)

In [160]:
# TMDB API Key
tmdb_secret = os.environ['TMDB_API']

# OMDB API Key
omdb_secret = os.environ['OMDB_API']

In [175]:
def tmdb_request(tmdb_id):
    """
    Constructing an api request for TMDB (The Movie Database) based on a given tmdb id, 
    then returning the json file
    
    Parameters:
        tmdb_id: movie id from TMDB
        
    Returns:
        tmdb_data: json data on given id
    """
    # construct url
    try:
        url = 'https://api.themoviedb.org/3/movie/' + tmdb_id + '?&api_key=' + tmdb_secret

        payload = {}
        headers = {}

        # perform get request
        response = requests.get(url, headers=headers, data = payload)

        # process request into usable JSON file
        tmdb_response = json.loads(response.text)
        tmdb_data.append(tmdb_response)

    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)

    return

In [176]:
tmdb_data = []
tmdb_request('14396')

In [178]:
tmdb_data = []
def tmdb_req_loop():
    """
    Iterates through the movielens dataframe & runs an API request to tmdb for each movie id
           
    Returns:
        Complete json data for every id in our movie dataframe
    """
    
    num_rows = movies.shape[0]
    row_count = 0
    
    while row_count < num_rows:
        tmdb_request((movies.iloc[row_count,2]))
        row_count +=1

In [179]:
tmdb_req_loop()

Error Connecting: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error Connecting: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [180]:
tmdb_data

[{'adult': False,
  'backdrop_path': '/n7zm7vcZ5m2nSV9Scyh2axpY8d4.jpg',
  'belongs_to_collection': None,
  'budget': 21000000,
  'genres': [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}],
  'homepage': 'http://www.freedomwriters.com/',
  'id': 1646,
  'imdb_id': 'tt0463998',
  'original_language': 'en',
  'original_title': 'Freedom Writers',
  'overview': 'A young teacher inspires her class of at-risk students to learn tolerance, apply themselves, and pursue education beyond high school.',
  'popularity': 16.616,
  'poster_path': '/81AdeUQT99N0xPg3j6RVh0YGOTk.jpg',
  'production_companies': [{'id': 215,
    'logo_path': '/tQyeqkCj24krhY2W8qeAztcOS37.png',
    'name': 'Double Feature Films',
    'origin_country': 'US'},
   {'id': 216,
    'logo_path': None,
    'name': 'Jersey Films',
    'origin_country': 'US'},
   {'id': 20396,
    'logo_path': None,
    'name': 'Kernos Filmproduktionsgesellschaft & Company',
    'origin_country': ''}],
  'production_countries': [{'iso_316

In [181]:
tmdb_df_raw = pd.json_normalize(tmdb_data)

In [182]:
# exporting raw dataframe
tmdb_df_raw.to_csv('25k_data.csv', index=False)

In [183]:
tmdb_df_raw

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,success,status_code,status_message
0,False,/n7zm7vcZ5m2nSV9Scyh2axpY8d4.jpg,NaN,21000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",http://www.freedomwriters.com/,1646.0,tt0463998,en,Freedom Writers,...,False,7.924,1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,/6dvpZbFu1VOCIcY1qToRHmj8Aaq.jpg,NaN,47000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.happilyneverafterthefilm.com/,5393.0,tt0308353,en,Happily N'Ever After,...,False,4.923,371.0,141084.0,Happily N'Ever After Collection,/aPjLYxKIBys6dlxXMZrDi8VowQK.jpg,/fw6YfVOmK0tVtaSUGb3GvLzEmvK.jpg,NaN,NaN,NaN
2,False,/52RD1oMub4E2lojzsuNMRNJNtco.jpg,NaN,12000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",,14171.0,tt0763840,en,Home of the Brave,...,False,5.453,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,/cMMjICzBe5qTx1igspFYdX8B3o2.jpg,NaN,20000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",,14396.0,tt0462229,en,Code Name: The Cleaner,...,False,5.100,205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,/6d9zm4ds01BA2EmX8GmzNlmIvyi.jpg,NaN,13000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",,1931.0,tt0775539,en,Stomp the Yard,...,False,6.905,351.0,160727.0,Stomp the Yard Collection,/2qAqDHlxVS4wOf1Z8mBGpTgp7PB.jpg,/zuaYuIAgnCY7Y6nkSGGhPx8RlOV.jpg,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25336,False,/ekVWMz32hsrRuSLf5KTjg3PvcUa.jpg,NaN,0.0,"[{'id': 36, 'name': 'History'}]",,622831.0,tt10551150,zh,決勝時刻,...,False,5.700,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25337,False,/oXOcbJXXdYgaOvQFwY5qLLAXnhy.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,18733.0,tt1156447,ja,ハッピーフライト,...,False,5.800,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25338,False,/bkYYS6hcsF372VovX3WBMs52ofd.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,63813.0,tt1382339,ta,சந்தோஷ் சுப்ரமணியம்,...,False,7.024,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25339,False,/3kb5b8IQCX4vd3baNBoZqAboP41.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",,499546.0,tt6671244,nl,Wij,...,False,5.800,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
tmdb_df_raw['imdb_id']

0         tt0463998
1         tt0308353
2         tt0763840
3         tt0462229
4         tt0775539
            ...    
25336    tt10551150
25337     tt1156447
25338     tt1382339
25339     tt6671244
25340     tt6755366
Name: imdb_id, Length: 25341, dtype: object

let's try to scrape all of this

In [119]:
import html5lib
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [185]:
tmdb_df_raw['imdb_id']

0         tt0463998
1         tt0308353
2         tt0763840
3         tt0462229
4         tt0775539
            ...    
25336    tt10551150
25337     tt1156447
25338     tt1382339
25339     tt6671244
25340     tt6755366
Name: imdb_id, Length: 25341, dtype: object

In [195]:
from urllib.error import HTTPError

In [222]:
def scrape():
    """
    Runs list of imdb ids and gathers the box office data for each movie
    """
    movie_list = [str(a) for a in tmdb_df_raw['imdb_id']]
    df_list = []
    pages_remaining = tmdb_df_raw.shape[0]
    for movie in movie_list:
        try:
            url = 'https://www.boxofficemojo.com/title/' + movie + '/?ref_=bo_se_r_1'
            tables = pd.read_html(url)
            df = tables[0]
            df = df.iloc[[0]]
            df['imdb_id'] = movie
            df_list.append(df)
            pages_remaining = pages_remaining - 1
            print(f'{movie} Scraped // Films left to scrape = {pages_remaining}')
        except (ValueError):
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered ValueError // Films left to scrape = {pages_remaining}')
        except HTTPError as err:
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered {err} // Films left to scrape = {pages_remaining}')
        except URLError as err:
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered {err} // Films left to scrape = {pages_remaining}')
        except TimeoutError as err:
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered {err} // Films left to scrape = {pages_remaining}')    
        except requests.exceptions.RequestException as err:
            pages_remaining = pages_remaining - 1
            print(f'{movie} Encountered {err} // Films left to scrape = {pages_remaining}')
    
    
    main_df = pd.concat(df_list)
    main_df.to_csv('25k_movie_data.csv')

In [223]:
pd.options.mode.chained_assignment = None

In [218]:
from urllib.error import URLError

In [219]:
URLError

urllib.error.URLError

In [224]:
main_df = []
scrape()

tt0463998 Scraped // Films left to scrape = 25340
tt0308353 Scraped // Films left to scrape = 25339
tt0763840 Scraped // Films left to scrape = 25338
tt0462229 Scraped // Films left to scrape = 25337
tt0775539 Scraped // Films left to scrape = 25336
tt0344854 Scraped // Films left to scrape = 25335
tt0772193 Scraped // Films left to scrape = 25334
tt0426883 Scraped // Films left to scrape = 25333
tt0455960 Scraped // Films left to scrape = 25332
tt0398808 Scraped // Films left to scrape = 25331
tt0473308 Scraped // Films left to scrape = 25330
tt0397044 Scraped // Films left to scrape = 25329
tt0799949 Scraped // Films left to scrape = 25328
tt0425430 Scraped // Films left to scrape = 25327
tt0490084 Scraped // Films left to scrape = 25326
tt0367959 Scraped // Films left to scrape = 25325
tt0477051 Scraped // Films left to scrape = 25324
tt0382932 Scraped // Films left to scrape = 25323
tt0469263 Scraped // Films left to scrape = 25322
tt0259324 Scraped // Films left to scrape = 25321


tt0765443 Scraped // Films left to scrape = 25176
tt0283503 Scraped // Films left to scrape = 25175
tt0925248 Scraped // Films left to scrape = 25174
tt0480269 Scraped // Films left to scrape = 25173
tt0473188 Scraped // Films left to scrape = 25172
tt0432021 Scraped // Films left to scrape = 25171
tt0419984 Scraped // Films left to scrape = 25170
tt0452625 Scraped // Films left to scrape = 25169
tt0758758 Scraped // Films left to scrape = 25168
tt0492956 Scraped // Films left to scrape = 25167
tt0808357 Scraped // Films left to scrape = 25166
tt0800027 Scraped // Films left to scrape = 25165
tt0486640 Scraped // Films left to scrape = 25164
tt0484562 Scraped // Films left to scrape = 25163
tt0408839 Scraped // Films left to scrape = 25162
tt0480242 Scraped // Films left to scrape = 25161
tt0838221 Scraped // Films left to scrape = 25160
tt0498399 Scraped // Films left to scrape = 25159
tt0414055 Scraped // Films left to scrape = 25158
tt0465538 Scraped // Films left to scrape = 25157


tt1001562 Scraped // Films left to scrape = 25012
tt0467200 Scraped // Films left to scrape = 25011
tt0839980 Scraped // Films left to scrape = 25010
tt0425413 Scraped // Films left to scrape = 25009
tt1097256 Encountered ValueError // Films left to scrape = 25008
tt0854678 Scraped // Films left to scrape = 25007
tt0772168 Scraped // Films left to scrape = 25006
tt0463027 Scraped // Films left to scrape = 25005
tt0997047 Scraped // Films left to scrape = 25004
tt0443649 Scraped // Films left to scrape = 25003
tt0200465 Scraped // Films left to scrape = 25002
tt0483607 Scraped // Films left to scrape = 25001
tt0451079 Scraped // Films left to scrape = 25000
tt0808279 Scraped // Films left to scrape = 24999
tt0813547 Scraped // Films left to scrape = 24998
tt0416044 Scraped // Films left to scrape = 24997
tt0912599 Scraped // Films left to scrape = 24996
tt0811138 Scraped // Films left to scrape = 24995
tt0475998 Scraped // Films left to scrape = 24994
tt0848557 Scraped // Films left to 

tt1064932 Scraped // Films left to scrape = 24848
tt0816545 Scraped // Films left to scrape = 24847
tt0847897 Scraped // Films left to scrape = 24846
tt0829193 Scraped // Films left to scrape = 24845
tt0997188 Scraped // Films left to scrape = 24844
tt0985025 Scraped // Films left to scrape = 24843
tt0479528 Scraped // Films left to scrape = 24842
tt0822858 Scraped // Films left to scrape = 24841
tt0787523 Scraped // Films left to scrape = 24840
tt1210351 Scraped // Films left to scrape = 24839
tt0879843 Scraped // Films left to scrape = 24838
tt0373051 Scraped // Films left to scrape = 24837
tt0765476 Scraped // Films left to scrape = 24836
tt1050160 Scraped // Films left to scrape = 24835
tt0470679 Scraped // Films left to scrape = 24834
tt0472071 Scraped // Films left to scrape = 24833
tt0869977 Scraped // Films left to scrape = 24832
tt0492486 Scraped // Films left to scrape = 24831
tt0901475 Scraped // Films left to scrape = 24830
tt1024942 Scraped // Films left to scrape = 24829


tt1010048 Scraped // Films left to scrape = 24684
tt0830515 Scraped // Films left to scrape = 24683
tt1042499 Scraped // Films left to scrape = 24682
tt0430922 Scraped // Films left to scrape = 24681
tt0976060 Scraped // Films left to scrape = 24680
tt1183732 Scraped // Films left to scrape = 24679
tt1124394 Scraped // Films left to scrape = 24678
tt0456340 Scraped // Films left to scrape = 24677
tt1130988 Scraped // Films left to scrape = 24676
tt0850669 Scraped // Films left to scrape = 24675
tt1016290 Scraped // Films left to scrape = 24674
tt0933033 Scraped // Films left to scrape = 24673
tt0772181 Scraped // Films left to scrape = 24672
tt0772176 Scraped // Films left to scrape = 24671
tt1064953 Scraped // Films left to scrape = 24670
tt1055366 Scraped // Films left to scrape = 24669
tt1132626 Scraped // Films left to scrape = 24668
tt0993789 Scraped // Films left to scrape = 24667
tt1135985 Scraped // Films left to scrape = 24666
tt1111948 Scraped // Films left to scrape = 24665


...

tt7169514 Scraped // Films left to scrape = 728
tt9507276 Scraped // Films left to scrape = 727
tt7442160 Scraped // Films left to scrape = 726
tt2075318 Scraped // Films left to scrape = 725
tt5001456 Scraped // Films left to scrape = 724
tt7255568 Scraped // Films left to scrape = 723
tt9652782 Encountered [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond // Films left to scrape = 722
tt10516972 Scraped // Films left to scrape = 721
tt6793818 Scraped // Films left to scrape = 720
tt8888186 Scraped // Films left to scrape = 719
tt5258106 Scraped // Films left to scrape = 718
tt10380066 Scraped // Films left to scrape = 717
tt3864056 Scraped // Films left to scrape = 716
tt9585788 Encountered ValueError // Films left to scrape = 715
tt7318102 Scraped // Films left to scrape = 714
tt9647768 Scraped // Films left to scrape = 713
tt6869820 S

tt7967412 Scraped // Films left to scrape = 563
tt6836936 Scraped // Films left to scrape = 562
tt10011448 Scraped // Films left to scrape = 561
tt4742618 Scraped // Films left to scrape = 560
tt6535880 Scraped // Films left to scrape = 559
tt6570372 Encountered ValueError // Films left to scrape = 558
tt3313740 Scraped // Films left to scrape = 557
nan Encountered HTTP Error 404: Not Found // Films left to scrape = 556
tt4586114 Scraped // Films left to scrape = 555
tt0454941 Scraped // Films left to scrape = 554
tt1240941 Scraped // Films left to scrape = 553
tt7416602 Scraped // Films left to scrape = 552
tt9252508 Scraped // Films left to scrape = 551
tt8359822 Scraped // Films left to scrape = 550
tt10122392 Scraped // Films left to scrape = 549
tt8860946 Scraped // Films left to scrape = 548
tt1047890 Scraped // Films left to scrape = 547
tt7080138 Scraped // Films left to scrape = 546
tt10550884 Scraped // Films left to scrape = 545
tt9358200 Scraped // Films left to scrape = 54

tt9358228 Scraped // Films left to scrape = 395
tt4843040 Scraped // Films left to scrape = 394
tt7210426 Scraped // Films left to scrape = 393
tt4520988 Scraped // Films left to scrape = 392
tt4647410 Scraped // Films left to scrape = 391
tt1985249 Scraped // Films left to scrape = 390
tt1086781 Scraped // Films left to scrape = 389
tt7013830 Scraped // Films left to scrape = 388
tt8809652 Scraped // Films left to scrape = 387
tt6112524 Scraped // Films left to scrape = 386
tt11043546 Scraped // Films left to scrape = 385
tt3879066 Encountered ValueError // Films left to scrape = 384
tt6494556 Scraped // Films left to scrape = 383
tt1193622 Scraped // Films left to scrape = 382
tt5120640 Scraped // Films left to scrape = 381
tt3382148 Scraped // Films left to scrape = 380
tt3800894 Scraped // Films left to scrape = 379
tt4029868 Encountered ValueError // Films left to scrape = 378
tt4363984 Scraped // Films left to scrape = 377
tt4625196 Scraped // Films left to scrape = 376
tt4659548

tt8106562 Scraped // Films left to scrape = 228
tt8383656 Scraped // Films left to scrape = 227
tt10326928 Scraped // Films left to scrape = 226
tt5368108 Scraped // Films left to scrape = 225
tt8176054 Scraped // Films left to scrape = 224
tt8991322 Scraped // Films left to scrape = 223
tt5305246 Scraped // Films left to scrape = 222
tt4169146 Scraped // Films left to scrape = 221
tt6141246 Scraped // Films left to scrape = 220
tt5087462 Scraped // Films left to scrape = 219
tt10516086 Scraped // Films left to scrape = 218
tt9648078 Scraped // Films left to scrape = 217
tt9563632 Scraped // Films left to scrape = 216
tt6450804 Scraped // Films left to scrape = 215
tt10971520 Scraped // Films left to scrape = 214
tt2440036 Scraped // Films left to scrape = 213
tt6211538 Scraped // Films left to scrape = 212
tt4163264 Scraped // Films left to scrape = 211
tt5658296 Scraped // Films left to scrape = 210
tt9817070 Scraped // Films left to scrape = 209
tt1933552 Scraped // Films left to sc

tt6924650 Scraped // Films left to scrape = 59
tt6271820 Scraped // Films left to scrape = 58
tt10675136 Scraped // Films left to scrape = 57
tt5563334 Scraped // Films left to scrape = 56
tt6494358 Encountered ValueError // Films left to scrape = 55
tt10553910 Scraped // Films left to scrape = 54
tt9806192 Scraped // Films left to scrape = 53
tt7275840 Scraped // Films left to scrape = 52
tt4631810 Scraped // Films left to scrape = 51
tt3730162 Scraped // Films left to scrape = 50
tt3660094 Scraped // Films left to scrape = 49
tt2050492 Scraped // Films left to scrape = 48
tt10310222 Scraped // Films left to scrape = 47
tt6463320 Scraped // Films left to scrape = 46
nan Encountered HTTP Error 404: Not Found // Films left to scrape = 45
tt8096832 Scraped // Films left to scrape = 44
tt10627776 Scraped // Films left to scrape = 43
tt6398496 Scraped // Films left to scrape = 42
nan Encountered HTTP Error 404: Not Found // Films left to scrape = 41
tt1125192 Scraped // Films left to scrap

In [256]:
# Load dataset
tmdb_df_raw = pd.read_csv('25k_data.csv')

In [257]:
tmdb_df_raw.iloc[0,7]

'tt0463998'

In [261]:
def omdb_request(imdb_id):
    """
    Constructing an api request for OMDB (the Open Movie Database) based on a given id, 
    then returning the json file
    
    Parameters:
        imdb_id: movie id from IMDB
        
    Returns:
        omdb_data: json data on given id
    """
    try:
        # construct url
        url = 'http://www.omdbapi.com/?apikey=' + omdb_secret + '&i=' + imdb_id

        payload = {}
        headers = {}

        # perform get request
        response = requests.get(url, headers=headers, data = payload)

        # process request into usable JSON file
        omdb_response = json.loads(response.text)
        omdb_data.append(omdb_response)
        
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)

    return

In [262]:
omdb_data = []
def omdb_req_loop():
    """
    Iterates through the tmdb dataframe & runs an API request to omdb for each movie id
           
    Returns:
        Complete json data for every id in our movie dataframe
    """
    
    num_rows = tmdb_df_raw.shape[0]
    row_count = 0
    
    while row_count < num_rows:
        omdb_request((tmdb_df_raw.iloc[row_count,7]))
        row_count +=1

In [263]:
omdb_req_loop()

Error Connecting: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error Connecting: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error Connecting: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error Connecting: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


In [ ]:
omdb_data

In [264]:
omdb_df_raw = pd.json_normalize(omdb_data)

In [265]:
# exporting raw dataframe
omdb_df_raw.to_csv('25k_omdb_raw_data.csv', index=False)

In [1]:
omdb_df_raw.head()

NameError: name 'omdb_df_raw' is not defined